In [31]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px

# sdt_path = ".."
# if sdt_path not in sys.path:
#     sys.path.insert(1, sdt_path)
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

from functools import partial
from tqdm import tqdm

import utils

from bayesian_github.bayesian_changepoint_detection.priors import const_prior
from bayesian_github.bayesian_changepoint_detection.bayesian_models import online_changepoint_detection
import bayesian_github.bayesian_changepoint_detection.online_likelihoods as online_ll
from bayesian_github.bayesian_changepoint_detection.hazard_functions import constant_hazard

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# data load

In [3]:
from glob import glob

#path = glob(os.path.join('..', 'Data', 'NXP', '*.csv'))
path = glob(os.path.join('..', 'Data', 'case*.csv'))

data = utils.load_data(path)

In [3]:
path

['..\\Data\\case06.csv',
 '..\\Data\\case07.csv',
 '..\\Data\\case08.csv',
 '..\\Data\\case13.csv',
 '..\\Data\\case14.csv',
 '..\\Data\\case15.csv',
 '..\\Data\\case16.csv',
 '..\\Data\\case18.csv',
 '..\\Data\\case19.csv',
 '..\\Data\\case21.csv',
 '..\\Data\\case22.csv',
 '..\\Data\\case23.csv',
 '..\\Data\\case24.csv',
 '..\\Data\\case25.csv',
 '..\\Data\\case26.csv',
 '..\\Data\\case32.csv',
 '..\\Data\\case34.csv',
 '..\\Data\\case35.csv',
 '..\\Data\\case36.csv',
 '..\\Data\\case37.csv']

# Label list

In [6]:
label_list = utils.load_label_list()

# online full example

In [1]:
utils.draw_online_example(data, path, num_row=4)

NameError: name 'utils' is not defined

# online grid search

In [14]:
param_grid = {'past': [3],
              'threshold': np.arange(0.1, 1, 0.5),
              'normalize': [2],
              'kappa': np.arange(0, 1.1, 0.5),
              'alpha': [0.1],
              'beta': [0.1],
              'mu': [0]}

In [15]:
grid = utils.grid_search(data, label_list, param_grid)

20it [00:05,  3.97it/s]


# score aggregation

In [28]:
example1 = pd.DataFrame(grid[1])
example = example1.copy()
example['cos'] = 0
cv_result = []
for case in grid:
    cv_temp = pd.DataFrame(case).fillna(0)
    #cv_temp = cv_temp.iloc[:,[0,1,3,4,5,2]]
    example['cos'] = cv_temp['cos'].values + example['cos'].values
    cv_result.append(cv_temp)
    
example['cos'] = example['cos'] / len(grid)

In [29]:
example.sort_values(by='cos', ascending=False)

,past,threshold,norm,kappa,alpha,beta,mu,cos
2,3,0.1,2,0.5,0.1,0.1,0,0.443206
5,3,0.6,2,1.0,0.1,0.1,0,0.443145
3,3,0.6,2,0.5,0.1,0.1,0,0.438676
4,3,0.1,2,1.0,0.1,0.1,0,0.419061
0,3,0.1,2,0.0,0.1,0.1,0,0.000000
1,3,0.6,2,0.0,0.1,0.1,0,0.000000


In [2]:
fig = px.parallel_coordinates(example.copy(),
                              color="cos",
                              labels={'cos':'score'},
                              title='cosine sim. calculation',
                              color_continuous_scale=px.colors.diverging.Tealrose)
                              
                              #color_continuous_scale=px.colors.sequential.Viridis
fig.show()

NameError: name 'px' is not defined